In [ ]:
# install slim version (default)
!pip install pycaret

In [ ]:
import pandas as pd
data=pd.read_csv('../input/heart-disease-prediction/heart_v2.csv')
data.head()

In [ ]:
data.nunique()

In [ ]:
data.skew()

In [ ]:
data.info()

In [ ]:
# initialize setup
from pycaret.classification import *
s = setup(data, target = 'heart disease',
         normalize = True,
          remove_multicollinearity = True,
          combine_rare_levels = True,
          ignore_low_variance = True,
          fix_imbalance = True,
          transformation = True
         )

# Available Models

In [ ]:
# check all the available models
models()

# Model Training & Selection

In [ ]:
# train Xgboost Classifier
xgboost = create_model('xgboost')

In [ ]:
print(xgboost)

In [ ]:
# train multiple models
multiple_models = [create_model(i) for i in ['dt', 'lr', 'xgboost']]

In [ ]:
# check multiple_models
type(multiple_models), len(multiple_models)

In [ ]:
print(multiple_models)

In [ ]:
# compare all models
best_model = compare_models()

In [ ]:
# check the best model
print(best_model)

In [ ]:
# predict on hold-out
pred_holdout = predict_model(best_model)

In [ ]:
# create copy of data drop target column
data2 = data.copy()
data2.drop('heart disease', axis=1, inplace=True)

In [ ]:
# generate predictions
predictions = predict_model(best_model, data = data2)

In [ ]:
# install gplearn
!pip install gplearn

In [ ]:
# import untrained estimator
from gplearn.genetic import SymbolicClassifier
sc = SymbolicClassifier()

In [ ]:
# train using create_model
sc_trained = create_model(sc)

In [ ]:
print(sc_trained)

In [ ]:
# check hold-out score
pred_holdout_sc = predict_model(sc_trained)

In [ ]:
# install ngboost
!pip install ngboost

In [ ]:
# import untrained estimator
from ngboost import NGBClassifier
ng = NGBClassifier()

In [ ]:
# train using create_model
ng_trained = create_model(ng)

In [ ]:
print(ng_trained)

In [ ]:
# create custom estimator
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances
class MyOwnModel(BaseEstimator, ClassifierMixin):

    def __init__(self, demo_param='demo'):
        self.demo_param = demo_param

    def fit(self, X, y):

        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y
        # Return the classifier
        return self

    def predict(self, X):

        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
        X = check_array(X)

        closest = np.argmin(euclidean_distances(X, self.X_), axis=1)
        return self.y_[closest]

In [ ]:
# import MyOwnModel class
mom = MyOwnModel()

In [ ]:
# train using create_model
mom_trained = create_model(mom)

In [ ]:
# generate predictions on data
predictions = predict_model(mom_trained, data=data)
predictions